In [429]:
from selenium import webdriver
from datetime import datetime
import pandas as pd
import numpy as np
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException


# Set up the Selenium WebDriver (make sure to have the appropriate webdriver installed)
driver = webdriver.Edge(executable_path=r"C:\Program Files (x86)\msedgedriver.exe")


# URL of the webpage containing the dropdown menu
url = f"https://www.basketball-reference.com/leagues/NBA_2024_per_game.html"
driver.get(url)

# Player Ratings

In [431]:
# Have the table only show accurate data
hide_partial_row_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="per_game_stats_toggle_partial_table"]'))
)
hide_partial_row_button.click()
print('Hide rows with partial data')

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (640, 904)
  (Session info: MicrosoftEdge=120.0.2210.91)


In [ ]:
# Get table with data
table = driver.find_element(By.XPATH,'//*[@id="per_game_stats"]')

# Get table header
headers = table.find_element(By.XPATH, '//*[@id="per_game_stats"]/thead').text.split(' ')

# Get table data
data = [row.text.split(' ') for row in table.find_elements(By.TAG_NAME, 'tr')][1:]

In [ ]:
# Remove 'Player' column from list
headers.pop(1)

# Add 'First Name' to list
headers.insert(1, 'First Name')

# Add 'Last Name' to list
headers.insert(2, 'Last Name')

In [ ]:
# Create a pandas Data Frame
player_df = pd.DataFrame(data=data)

# Drop last column
player_df = player_df.drop(player_df.columns[31],axis=1)

# Add headers
player_df.columns = headers

In [ ]:
player_df.head()

In [ ]:
player_df.info()

In [ ]:
# Remove rows with any null values'
player_df = player_df.dropna(axis=0, how='any')

In [ ]:
player_df.info()

In [ ]:
# Convert columns with data type object to float
player_df = player_df.apply(pd.to_numeric, errors='ignore', downcast='float')
player_df.dtypes

In [ ]:
player_df.head()

# Team Ratings

In [499]:
from selenium import webdriver
from datetime import datetime
import pandas as pd
import numpy as np
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException

# Set up the Selenium WebDriver (make sure to have the appropriate webdriver installed)
driver = webdriver.Edge(executable_path=r"C:\Program Files (x86)\msedgedriver.exe")

# URL of the webpage containing the dropdown menu
url = f"https://www.basketball-reference.com/leagues/NBA_2024_ratings.html"
driver.get(url)

In [500]:
# Extract table
table = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="ratings"]'))
)

# Get headers used in table
thead = table.find_element(By.TAG_NAME, 'thead')
headers = [header.text.split() for header in thead.find_elements(By.TAG_NAME,'tr')][1]

# Get data from table
tbody = table.find_element(By.XPATH, '//*[@id="ratings"]/tbody')
rows = [row.text.split() for row in tbody.find_elements(By.TAG_NAME, 'tr')]

In [510]:
team_names = []

# Extract team names from table
names = tbody.find_elements(By.CLASS_NAME, 'left')

# Extract team names and add to array
for name in names:
    if name.text == 'Team':
        continue
    else:
        team_names.append(name.text)

team_names

['Boston Celtics',
 'Oklahoma City Thunder',
 'Philadelphia 76ers',
 'Minnesota Timberwolves',
 'Los Angeles Clippers',
 'New Orleans Pelicans',
 'Denver Nuggets',
 'New York Knicks',
 'Milwaukee Bucks',
 'Indiana Pacers',
 'Houston Rockets',
 'Cleveland Cavaliers',
 'Orlando Magic',
 'Sacramento Kings',
 'Dallas Mavericks',
 'Miami Heat',
 'Golden State Warriors',
 'Phoenix Suns',
 'Los Angeles Lakers',
 'Atlanta Hawks',
 'Toronto Raptors',
 'Brooklyn Nets',
 'Chicago Bulls',
 'Utah Jazz',
 'Memphis Grizzlies',
 'Portland Trail Blazers',
 'San Antonio Spurs',
 'Washington Wizards',
 'Charlotte Hornets',
 'Detroit Pistons']

In [511]:
table_data = []

for row in rows:
    # Find rows where the team name has two parts 'Boston Celtics'
    if len(row) == 16:
        row.pop(1)
        row.pop(1)
    # Find rows where the team name has three parts 'Oklahoma City Thunder'
    elif len(row) == 17:
        row.pop(1)
        row.pop(1)
        row.pop(1)
    elif row[0] == 'Unadjusted' or row[0] == 'Rk':
        continue  # Skip headers and other irrelevant rows
    table_data.append(row)
    
table_data

[['1',
  'E',
  'A',
  '29',
  '8',
  '.784',
  '10.35',
  '122.93',
  '112.36',
  '10.58',
  '10.96',
  '123.10',
  '111.92',
  '11.18'],
 ['2',
  'W',
  'NW',
  '25',
  '11',
  '.694',
  '7.58',
  '122.24',
  '114.61',
  '7.63',
  '7.86',
  '122.44',
  '114.52',
  '7.92'],
 ['3',
  'E',
  'A',
  '23',
  '13',
  '.639',
  '7.81',
  '120.61',
  '112.78',
  '7.83',
  '7.10',
  '120.03',
  '112.89',
  '7.14'],
 ['4',
  'W',
  'NW',
  '26',
  '11',
  '.703',
  '5.38',
  '115.60',
  '110.14',
  '5.46',
  '6.64',
  '116.01',
  '109.28',
  '6.73'],
 ['5',
  'W',
  'P',
  '24',
  '13',
  '.649',
  '5.70',
  '120.79',
  '115.03',
  '5.76',
  '5.30',
  '120.56',
  '115.20',
  '5.36'],
 ['6',
  'W',
  'SW',
  '23',
  '15',
  '.605',
  '5.00',
  '118.35',
  '113.23',
  '5.12',
  '4.92',
  '118.83',
  '113.80',
  '5.02'],
 ['7',
  'W',
  'NW',
  '26',
  '13',
  '.667',
  '4.95',
  '120.04',
  '114.88',
  '5.15',
  '4.78',
  '120.31',
  '115.34',
  '4.97'],
 ['8',
  'E',
  'A',
  '22',
  '15',
  '.

In [512]:
# Add respective team names to corresponding table data rows
for i in range(len(table_data)):
    table_data[i].insert(1, team_names[i])

In [513]:
table_data

[['1',
  'Boston Celtics',
  'E',
  'A',
  '29',
  '8',
  '.784',
  '10.35',
  '122.93',
  '112.36',
  '10.58',
  '10.96',
  '123.10',
  '111.92',
  '11.18'],
 ['2',
  'Oklahoma City Thunder',
  'W',
  'NW',
  '25',
  '11',
  '.694',
  '7.58',
  '122.24',
  '114.61',
  '7.63',
  '7.86',
  '122.44',
  '114.52',
  '7.92'],
 ['3',
  'Philadelphia 76ers',
  'E',
  'A',
  '23',
  '13',
  '.639',
  '7.81',
  '120.61',
  '112.78',
  '7.83',
  '7.10',
  '120.03',
  '112.89',
  '7.14'],
 ['4',
  'Minnesota Timberwolves',
  'W',
  'NW',
  '26',
  '11',
  '.703',
  '5.38',
  '115.60',
  '110.14',
  '5.46',
  '6.64',
  '116.01',
  '109.28',
  '6.73'],
 ['5',
  'Los Angeles Clippers',
  'W',
  'P',
  '24',
  '13',
  '.649',
  '5.70',
  '120.79',
  '115.03',
  '5.76',
  '5.30',
  '120.56',
  '115.20',
  '5.36'],
 ['6',
  'New Orleans Pelicans',
  'W',
  'SW',
  '23',
  '15',
  '.605',
  '5.00',
  '118.35',
  '113.23',
  '5.12',
  '4.92',
  '118.83',
  '113.80',
  '5.02'],
 ['7',
  'Denver Nuggets',


NameError: name 'table_daat' is not defined

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [294]:
team_ratings = pd.DataFrame(data=rows)
team_ratings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,Boston,Celtics,E,A,28,8,.778,10.44,122.90,112.22,10.68,10.91,122.87,111.73,11.14,None
1,2,Oklahoma,City,Thunder,W,NW,24,11,.686,7.57,122.10,114.48,7.62,7.87,122.29,114.36,7.93
2,3,Philadelphia,76ers,E,A,23,12,.657,8.23,120.71,112.47,8.23,7.54,120.26,112.70,7.56,None
3,4,Minnesota,Timberwolves,W,NW,25,10,.714,5.29,115.55,110.20,5.35,6.11,115.68,109.50,6.18,None
4,5,Denver,Nuggets,W,NW,26,12,.684,5.42,120.26,114.62,5.63,5.38,120.69,115.12,5.57,None
5,6,Los,Angeles,Clippers,W,P,23,13,.639,5.64,120.39,114.71,5.69,5.23,120.17,114.89,5.28
6,7,New,Orleans,Pelicans,W,SW,22,15,.595,4.16,117.68,113.41,4.27,3.91,118.14,114.15,3.99
7,8,New,York,Knicks,E,A,21,15,.583,3.33,119.37,115.84,3.54,3.59,119.09,115.29,3.80
8,9,Milwaukee,Bucks,E,C,25,12,.676,4.22,121.74,117.63,4.12,3.15,120.71,117.64,3.08,None
9,10,Indiana,Pacers,E,C,21,15,.583,2.64,123.86,121.24,2.62,2.71,124.12,121.42,2.70,None


In [250]:
team_ratings = team_ratings.drop([15, 16])

In [251]:
team_ratings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,Boston,Celtics,E,A,27,7,.794,10.62,122.87,112.01,10.86,11.03,123.11,111.84,11.27,None
1,2,Oklahoma,City,Thunder,W,NW,23,11,.676,7.56,121.73,114.12,7.61,8.27,122.13,113.79,8.33
2,3,Philadelphia,76ers,E,A,23,11,.676,8.79,121.08,112.28,8.80,8.25,120.72,112.45,8.27,None
3,4,Minnesota,Timberwolves,W,NW,25,9,.735,5.65,115.61,109.89,5.73,6.34,115.71,109.29,6.43,None
4,5,Denver,Nuggets,W,NW,25,12,.676,5.11,119.87,114.56,5.31,5.33,120.44,114.93,5.51,None
5,6,Los,Angeles,Clippers,W,P,22,12,.647,5.26,120.09,114.81,5.28,4.75,119.82,115.05,4.78
6,7,Milwaukee,Bucks,E,C,25,10,.714,5.03,122.16,117.22,4.94,3.96,120.98,117.09,3.89,None
7,8,New,York,Knicks,E,A,20,15,.571,2.97,119.32,116.13,3.18,3.65,119.22,115.36,3.86
8,9,New,Orleans,Pelicans,W,SW,21,15,.583,3.36,117.02,113.61,3.42,3.02,117.48,114.42,3.06
9,10,Houston,Rockets,W,SW,17,16,.515,2.55,114.80,112.21,2.59,2.78,114.96,112.15,2.82,None
